In [ ]:
pwd

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import sys
import pathlib
import requests
import json
from nyoka import PMML43Ext as ny
# from nyokaBase.skl.pmml_to_skl import pmml_to_skl
fPath='C:/Users/swsh/Desktop/nyokaDurr/nyoka_deurr/nyoka-43Ext/'

from nyoka.reconstruct.pmml_to_pipeline_model import generate_skl_model

Using TensorFlow backend.


In [42]:
ppp=pathlib.Path(fPath+'UseCaseDemo.pmml')
ppp.name,ppp.suffix

('UseCaseDemo.pmml', '.pmml')

In [43]:
pmmlObj=ny.parse(fPath+'UseCase1Self.pmml',silence=True)

In [44]:
ppp=pmmlObj.__dict__['DeepNetwork'][0]#.__dict__

In [45]:
import ast

In [6]:
modelObjectToCheck=['AssociationModel', 'AnomalyDetectionModel', 'BayesianNetworkModel',
                    'BaselineModel', 'ClusteringModel', 'DeepNetwork', 'GaussianProcessModel',
                    'GeneralRegressionModel', 'MiningModel', 'NaiveBayesModel', 'NearestNeighborModel', 
                    'NeuralNetwork', 'RegressionModel', 'RuleSetModel', 'SequenceModel', 'Scorecard', 
                    'SupportVectorMachineModel', 'TextModel', 'TimeSeriesModel', 'TreeModel']

In [7]:
def getCode(strngVal):
    lines = []
    code = strngVal.lstrip('\n')
    leading_spaces = len(code) - len(code.lstrip(' '))
    for line in code.split('\n'):
        lines.append(line[leading_spaces:])
    code = '\n'.join(lines)
    return code

def getCodeObjectToProcess(codeVal):
    d = {}
    exec(codeVal, None,d)
    objeCode=d[list(d.keys())[0]]
    return objeCode

def getTargetAndColumnsName(modObjToDetect):
    targetCol=None
    listOFColumns=[]
    if modObjToDetect.__dict__['original_tagname_']=='MiningModel':
        for minF in modObjToDetect.get_MiningSchema().__dict__['MiningField']:
            if minF.__dict__['usageType'] == 'target':
                targetCol=minF.__dict__['name']
            else:
                listOFColumns.append(minF.__dict__['name'])
    else:
        print ('Add support')
        targetCol=None
        listOFColumns=[]
    return listOFColumns,targetCol



In [8]:
def nyObjOfModel(pmmlObj,singMod):
    import nyoka.PMML43Ext as ny
    if singMod['pmmlModelObject'].__dict__['original_tagname_']=='MiningModel':
        nyokaObj=ny.PMML(MiningBuildTask=pmmlObj.MiningBuildTask,DataDictionary=pmmlObj.DataDictionary,MiningModel=[singMod['pmmlModelObject']])
    elif singMod['pmmlModelObject'].__dict__['original_tagname_']=='DeepNetwork':
        nyokaObj=ny.PMML(DataDictionary=pmmlObj.DataDictionary,DeepNetwork=[singMod['pmmlModelObject']])
    else:
        nyokaObj=None
    return nyokaObj

In [9]:
modelObj=[]
for inMod in modelObjectToCheck:
    if len(pmmlObj.__dict__[inMod]) >0:
        modPMMLObj=pmmlObj.__dict__[inMod]
        if inMod == 'DeepNetwork':
            for ininMod in modPMMLObj:
                modelObj.append({'modelArchType':'NNModel','pmmlModelObject':ininMod,'recoModelObj':None,'listOFColumns':None,'targetCol':None})
        else:
            for ininMod in modPMMLObj:
                colInfo=getTargetAndColumnsName(ininMod)
#                 recoModelObj=generateModelfromPMML(ininMod)
                modelObj.append({'modelArchType':'SKLModel','pmmlModelObject':ininMod,'recoModelObj':None,'listOFColumns':colInfo[0],'targetCol':colInfo[1]})

In [10]:
modelObj

[{'modelArchType': 'NNModel',
  'pmmlModelObject': <nyoka.PMML43Ext.DeepNetwork at 0x252f0605b00>,
  'recoModelObj': None,
  'listOFColumns': None,
  'targetCol': None},
 {'modelArchType': 'SKLModel',
  'pmmlModelObject': <nyoka.PMML43Ext.MiningModel at 0x252f06286d8>,
  'recoModelObj': None,
  'listOFColumns': ['Open',
   'Close',
   'Low',
   'RollingOpen',
   'RollingClose',
   'RollingLow',
   'StdDevOpen',
   'StdDevClose',
   'StdDevLow'],
  'targetCol': 'Volume'}]

In [11]:
tempDict={}
tempDict['train']={}

tempDict['score']={}

for singMod in modelObj:
    if singMod['pmmlModelObject'].taskType=='trainAndscore':
        tempDict['train'][singMod['pmmlModelObject'].modelName]={}
        tempDict['train'][singMod['pmmlModelObject'].modelName]['modelObj']=singMod
        tempDict['train'][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlDdicObj']=pmmlObj.DataDictionary
        tempDict['train'][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlNyokaObj']=nyObjOfModel(pmmlObj,singMod)
        tempDict['score'][singMod['pmmlModelObject'].modelName]={}
        tempDict['score'][singMod['pmmlModelObject'].modelName]['modelObj']=singMod
        tempDict['score'][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlDdicObj']=pmmlObj.DataDictionary
        tempDict['train'][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlNyokaObj']=nyObjOfModel(pmmlObj,singMod)
    else:
        tempDict[singMod['pmmlModelObject'].taskType][singMod['pmmlModelObject'].modelName]={}
        tempDict[singMod['pmmlModelObject'].taskType][singMod['pmmlModelObject'].modelName]['modelObj']=singMod
        tempDict[singMod['pmmlModelObject'].taskType][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlDdicObj']=pmmlObj.DataDictionary
        tempDict[singMod['pmmlModelObject'].taskType][singMod['pmmlModelObject'].modelName]['modelObj']['pmmlNyokaObj']=nyObjOfModel(pmmlObj,singMod)

In [12]:
tempDict2={}
for taType in tempDict:
    print (taType)
    tempTa=list(tempDict[taType].keys())
    tempTa.sort()
    print (tempTa)
    for taTTemp in tempTa:
        print ('>>>>>  ',taTTemp)
        if taType in tempDict2:
            pass
        else:
            tempDict2[taType]={}
        tempDict2[taType][taTTemp]=tempDict[taType][taTTemp]

train
['model1']
>>>>>   model1
score
['model1', 'model2']
>>>>>   model1
>>>>>   model2


In [13]:
tempDict=tempDict2.copy()

In [14]:
for sc1 in pmmlObj.script:
#     print (sc1.class_,sc1.for_,sc1.scriptPurpose,sc1.valueOf_)
    if sc1.scriptPurpose=='trainAndscore':
        tempDict['train'][sc1.for_][sc1.class_]={}
        tempDict['train'][sc1.for_][sc1.class_+'_code']=getCode(sc1.valueOf_)
        tempDict['train'][sc1.for_][sc1.class_]=getCodeObjectToProcess(getCode(sc1.valueOf_))
        tempDict['score'][sc1.for_][sc1.class_]={}
        tempDict['score'][sc1.for_][sc1.class_+'_code']=getCode(sc1.valueOf_)
        tempDict['score'][sc1.for_][sc1.class_]=getCodeObjectToProcess(getCode(sc1.valueOf_))
    else:
        tempDict[sc1.scriptPurpose][sc1.for_][sc1.class_]={}
        tempDict[sc1.scriptPurpose][sc1.for_][sc1.class_+'_code']=getCode(sc1.valueOf_)
        tempDict[sc1.scriptPurpose][sc1.for_][sc1.class_]=getCodeObjectToProcess(getCode(sc1.valueOf_))

In [15]:
taskTypesName=list(tempDict.keys())
listOfModelNames=set([k for j in tempDict for k in tempDict[j]])

hyperParDict={}
for extObj in pmmlObj.MiningBuildTask.Extension:
    if extObj.name=='hyperparameters':
        hyperParDict[extObj.for_]=ast.literal_eval(extObj.value)

In [16]:
hyperParDict

{'model2': {'num_leaves': 31,
  'objective': 'regression',
  'metric': 'mean_absolute_error'}}

In [17]:
%load_ext autoreload
%reload_ext autoreload

In [18]:
modelLoadStatus=[]
for taskT in tempDict:
    print (taskT)
    for mO in tempDict[taskT]:
        if tempDict[taskT][mO]['modelObj']['modelArchType']=="NNModel":
            modelProp=tempDict[taskT][mO]['modelObj']['pmmlNyokaObj']
            recoModelObj=generate_skl_model(modelProp)
            if str(recoModelObj) != 'None':
                tempDict[taskT][mO]['modelObj']['recoModelObj']=recoModelObj
                modelLoadStatus.append(1)
            else:
                modelLoadStatus.append(0)
            try:
                tempDict[taskT][mO]['modelObj']['hyperparameters']=hyperParDict[mO]
            except:
                tempDict[taskT][mO]['modelObj']['hyperparameters']=None
        elif tempDict[taskT][mO]['modelObj']['modelArchType']=="SKLModel":
            modelProp=tempDict[taskT][mO]['modelObj']['pmmlNyokaObj']
            recoModelObj=generate_skl_model(modelProp)
            if recoModelObj != None:
                tempDict[taskT][mO]['modelObj']['recoModelObj']=recoModelObj
                modelLoadStatus.append(1)
            else:
                modelLoadStatus.append(0)
            try:
                tempDict[taskT][mO]['modelObj']['hyperparameters']=hyperParDict[mO]
            except:
                tempDict[taskT][mO]['modelObj']['hyperparameters']=None

train
score
Creating Layers: [###############---------------] 50% lstm_2                                       WARNING:tensorflow:From C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Creating Layers: [##############################] 100% dense_4                                     
Applying Weights: [##############################] 100% dense_4                                    


In [55]:
tempDict['train']

{'model1': {'modelObj': {'modelArchType': 'SKLModel',
   'pmmlModelObject': <nyoka.PMML43Ext.MiningModel at 0x252f06286d8>,
   'recoModelObj': <lightgbm.basic.Booster at 0x252f0c76160>,
   'listOFColumns': ['Open',
    'Close',
    'Low',
    'RollingOpen',
    'RollingClose',
    'RollingLow',
    'StdDevOpen',
    'StdDevClose',
    'StdDevLow'],
   'targetCol': 'Volume',
   'pmmlDdicObj': <nyoka.PMML43Ext.DataDictionary at 0x252f0605588>,
   'pmmlNyokaObj': <nyoka.PMML43Ext.PMML at 0x252f0c375f8>,
   'hyperparameters': None},
  'preprocessing': <function __main__.preProcessDataTrain()>,
  'preprocessing_code': "def preProcessDataTrain():\n    import pandas as pd\n    data=pd.read_csv('../ZMOD/Data/NIFTY.txt',header=None)\n    data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']\n    toData=data[['Open','Close','Low','Volume']]\n    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()\n    toData['RollingClose']=toData.rolling(window=5)[['Clo

## Train Model

In [19]:
import pandas as pd
import numpy as np
from sklearn import ensemble,metrics
from sklearn import datasets
import lightgbm as lgb

data=pd.read_csv('NIFTY.txt',header=None)
data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
toData=data[['Open','Close','Low','Volume']]

def preProcessDataScore(toData):
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

def preProcessDataTrain():
    import pandas as pd
    data=pd.read_csv('../ZMOD/Data/NIFTY.txt',header=None)
    data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
    toData=data[['Open','Close','Low','Volume']]
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

toData=preProcessDataScore(toData)

In [50]:
modltToTest=tempDict['train']['model1']['modelObj']['recoModelObj']

In [54]:
type(modltToTest)=='lightgbm.basic.Booster'


False

In [21]:
modltToTest.best_iteration

-1

In [22]:
XVar=['Open', 'Close', 'Low',  'RollingOpen', 'RollingClose','RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow']
YVar=['Volume']

In [23]:
train_data=lgb.Dataset(toData[XVar],toData[YVar])
num_round = 10
param = {'num_leaves': 31, 'objective': 'regression'}
param['metric'] = 'mean_absolute_error'
model1_ = lgb.train(param, train_data, num_round,init_model=modltToTest)

In [24]:
scor=modltToTest.predict(toData[XVar])

In [25]:
model1_.predict(toData[XVar])

array([30.97218658, 30.97218658, 30.97218658, ..., 20.82821035,
       20.82821035, 20.82821035])

In [26]:
scor

array([26.23067591, 26.23067591, 26.23067591, ..., 18.59628762,
       18.59628762, 18.59628762])

In [27]:
def preProcessData2nd(toData,scored):
    toData['resi']=toData['Volume']-scored
    toData['Rollingresi']=toData.rolling(window=5)[['resi']].mean()
    toData['StdDevresi']=toData.rolling(window=5)[['resi']].std()
    toData=toData.fillna(method='bfill')
    toData['Volume2']=toData['Volume'].apply(lambda x: 1 if x>17 else 0)
    return toData


toData2=preProcessData2nd(toData,scor)

In [28]:
toData2.head()

,Open,Close,Low,Volume,RollingOpen,RollingClose,RollingLow,StdDevOpen,StdDevClose,StdDevLow,resi,Rollingresi,StdDevresi,Volume2
0,10531.70,10531.70,10531.70,100,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,73.769324,39.969324,18.992104,1
1,10533.75,10535.50,10518.00,57,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,30.769324,39.969324,18.992104,1
2,10523.00,10527.50,10522.25,59,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,32.769324,39.969324,18.992104,1
3,10525.80,10526.15,10522.60,60,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,33.769324,39.969324,18.992104,1
4,10522.30,10522.95,10519.65,55,10527.31,10528.76,10522.84,5.164833,4.904768,5.303466,28.769324,39.969324,18.992104,1


In [29]:
XVar2=['resi', 'Rollingresi', 'StdDevresi']
YVar2=['Volume2']

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [31]:
kk=toData2[XVar2].shape[0]
hh=toData2[XVar2].shape[1]
XInde=toData2[XVar2].values.reshape(kk,1,hh)
YDepe=toData2[YVar2].values#.reshape(1,kk)

In [32]:
verbose, epochs, batch_size = 1, 5, 32
#n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(32, input_shape=(None,3)))
# model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XInde,YDepe, epochs=epochs, batch_size=32, verbose=verbose)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
92630/92630 [==============================] - 6s 70us/step - loss: 0.0114 - acc: 0.9968
Epoch 2/5
92630/92630 [==============================] - 6s 61us/step - loss: 0.0051 - acc: 0.9980
Epoch 3/5
92630/92630 [==============================] - 6s 61us/step - loss: 0.0048 - acc: 0.9980
Epoch 4/5
92630/92630 [==============================] - 6s 60us/step - loss: 0.0045 - acc: 0.9982
Epoch 5/5
92630/92630 [==============================] - 6s 62us/step - loss: 0.0046 - acc: 0.9982


In [33]:
model

In [34]:
from nyoka import model_to_pmml

## To export in ZMK

In [35]:
toExportDict={
    'model1':{
        'hyperparameters':None,
        'preProcessingScript':{'scripts':[preProcessDataTrain,preProcessDataScore], 'scriptpurpose':['train','score']},
        'pipelineObj':None,
        'modelObj':model1_,
        'featuresUsed':XVar,
        'targetName':'Volume',
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    },
    'model2':{
    'hyperparameters':param,
    'preProcessingScript':{'scripts':[preProcessData2nd], 'scriptpurpose':['score']},
    'pipelineObj':None,
    'modelObj':model,
    'featuresUsed':XVar2,
    'targetName':'Volume2',
    'postProcessingScript':{'scripts':[preProcessData2nd], 'scriptpurpose':['score']},
    'taskType': 'score'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="UseCase1Self.pmml")

In [36]:
taskTypesName=list(tempDict.keys())
listOfModelNames=set([k for j in tempDict for k in tempDict[j]])
listOfModelNames

toExportDict={}
for objDet in listOfModelNames:
    toExportDict[objDet]={'hyperparameters':None,
        'preProcessingScript':{'scripts':[], 'scriptpurpose':[]},
        'modelObj':None,
        'pipelineObj':None,
        'featuresUsed':None,
        'targetName':None,
        'postProcessingScript':None,
        'taskType': None}
    
for modObjeCom in tempDict:
    print (modObjeCom)
    if modObjeCom == 'train':
        for echMod in toExportDict:
            if echMod in tempDict[modObjeCom]:
                print ('>>>>>',echMod)
                toExportDict[echMod]['modelObj']=tempDict[modObjeCom][echMod]['modelObj']['recoModelObj']
                toExportDict[echMod]['preProcessingScript']['scripts'].append(tempDict[modObjeCom][echMod]['preprocessing_code'])
                toExportDict[echMod]['preProcessingScript']['scriptpurpose'].append(modObjeCom)
                toExportDict[echMod]['taskType']=modObjeCom
                toExportDict[echMod]['featuresUsed']=tempDict[modObjeCom][echMod]['modelObj']['listOFColumns']
                toExportDict[echMod]['targetName']=tempDict[modObjeCom][echMod]['modelObj']['targetCol']
                toExportDict[echMod]['hyperparameters']=tempDict[modObjeCom][echMod]['modelObj']['hyperparameters']
    if modObjeCom == 'score':
        for echMod in toExportDict:
            if echMod in tempDict[modObjeCom]:
                print ('>>>>',echMod)
                toExportDict[echMod]['modelObj']=tempDict[modObjeCom][echMod]['modelObj']['recoModelObj']
                toExportDict[echMod]['preProcessingScript']['scripts'].append(tempDict[modObjeCom][echMod]['preprocessing_code'])
                toExportDict[echMod]['preProcessingScript']['scriptpurpose'].append(modObjeCom)
                toExportDict[echMod]['targetName']=tempDict[modObjeCom][echMod]['modelObj']['targetCol']
                toExportDict[echMod]['hyperparameters']=tempDict[modObjeCom][echMod]['modelObj']['hyperparameters']
                
for modNa in listOfModelNames:
    if (modNa in tempDict['train']) & (modNa in tempDict['score']):
        toExportDict[modNa]['taskType']='trainAndscore'
    if ((modNa in tempDict['train'] )== False) & (modNa in tempDict['score']):
        toExportDict[modNa]['taskType']='score'

train
>>>>> model1
score
>>>> model2
>>>> model1


In [37]:
tempTa2=list(toExportDict.keys())
tempTa2.sort()
toExportReOrdered={}
for taTTemp2 in tempTa2:
    toExportReOrdered[taTTemp2]=toExportDict[taTTemp2]

In [38]:
toExportReOrdered

{'model1': {'hyperparameters': None,
  'preProcessingScript': {'scripts': ["def preProcessDataTrain():\n    import pandas as pd\n    data=pd.read_csv('../ZMOD/Data/NIFTY.txt',header=None)\n    data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']\n    toData=data[['Open','Close','Low','Volume']]\n    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()\n    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()\n    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()\n    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()\n    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()\n    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()\n    toData=toData.fillna(method='bfill')\n    return toData\n",
    "def preProcessDataScore(toData):\n    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()\n    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()\n    toData['RollingLo

In [40]:
from nyoka.skl.skl_to_pmml_V1 import model_to_pmml

In [41]:
conMod=model_to_pmml(toExportDict, pmml_f_name='from_sklearn.pmml')

model_name >>>>>>>>  model2
model_name >>>>>>>>  model1


In [ ]:
conMod.export(open('outputPMML.pmml','w'),0)

In [ ]:
conMod.MiningModel[0].MiningSchema.MiningField[-2].name

In [ ]:
testD=pd.DataFrame([{'MedInc': 8.3252,
 'HouseAge': 41.0,
 'AveRooms': 6.984126984126984,
 'AveBedrms': 1.0238095238095237,
 'Population': 322.0,
 'AveOccup': 2.5555555555555554,
 'Latitude': 37.88,
 'Longitude': -122.23,'ExtraColumn':200}])

toExportDict['model1']['pipelineObj'].predict(testD)

In [ ]:
testData={'MedInc': 8.3252,
 'HouseAge': 41.0,
 'AveRooms': 6.984126984126984,
 'AveBedrms': 1.0238095238095237,
 'Population': 322.0,
 'AveOccup': 2.5555555555555554,
 'Latitude': 37.88,
 'Longitude': -122.23}

In [ ]:
import requests
import json
url='http://localhost:8000/api/v1/newloadmodels'
para={'filePath':fPath+'rf_housing_model.pmml'}
reas=requests.post(url,data=para)
reas.text

In [ ]:
import pandas as pd

In [ ]:
preproc=tempDict['score']['model1']['preprocessing'](testData)

In [ ]:
list(tempDict['score'].keys())

In [ ]:
tempDict['score']['model1']['modelObj']['recoModelObj']

In [ ]:
tempDict['score']['model1']['preprocessing']

In [ ]:
tempDict#['train']['Model1']['preprocessing'](6)

# Calling rest Calls

In [ ]:
import requests
import json
url='http://localhost:8000/api/v1/newloadmodels'
para={'filePath':fPath+'useCase2.pmml'}
reas=requests.post(url,data=para)
reas.text

In [ ]:
url='http://10.60.27.137/zmk/api/v1/runningTasks'
reas=requests.get(url)#,data=para)
reas.text

In [ ]:
para={'filePath':fPath+'useCase2.pmml'}
reas=requests.get(url,data=para)
reas.text

In [ ]:
# para={'filePath':fPath+'useCase1.pmml'}
# reas=requests.post(url,data=para)
# reas.text

In [ ]:
# from types import FunctionType
# def kk(c):
#     return c
# pp=kk.__code__
# print(pp)

# oo=FunctionType(pp,globals())
# print(oo)

# kk(3)

In [ ]:
# def postProcessing(x):
#     path=x+x
#     print ('Done')
#     return path

# pp=postProcessing.__code__
# print(pp)

# oo=FunctionType(pp,globals())
# print(oo)

# oo(3)

In [ ]:
# coda=compile(tempDict['train']['Model1']['preprocessing'],"<script>", "exec")
# dataCode = marshal.dumps(coda)
# kk=marshal.loads(dataCode)
# kk(2)

In [ ]:
import shutil

In [ ]:
tocheck=fPath+'rf_housing_model.pmml'

In [ ]:
import os

In [ ]:
os.path.exists(tocheck)

In [ ]:
from itertools import count

fName=next(filename)
filePath=fPath+fName+'.pmml'
while os.path.exists(filePath)==False:
    fName=next(filename)
    print (fName)

In [ ]:
filePath

In [ ]:
# from itertools import count

next(filename)

In [ ]:
fName="rf_housing_model_V2"

In [ ]:
increName(fName)

In [ ]:
fName